Jupyter notebook to extract features from the 2D segmentation
Written by Dominik Waibel & Niklas Kiermeyer

In [ ]:
#import dependencies
import os 
import numpy as np
from skimage.io import imread, imsave
from skimage.measure import label, regionprops
from skimage import measure
import trimesh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import trimesh
import cv2
import math
from skimage.transform import resize
import numpy.linalg as linalg
from pyellipsoid import drawing
from skimage.filters import threshold_otsu
from skimage.feature import shape_index
from scipy.ndimage.measurements import center_of_mass
from skimage.filters import gaussian
import seaborn as sns
import csv
from scipy.ndimage.morphology import binary_dilation
import copy
from scipy.ndimage import gaussian_filter
from skimage.morphology import convex_hull_image
from skimage.feature import greycomatrix, greycoprops
from skimage.measure import moments
from skimage.feature import hog
from skimage.filters import gabor

In [ ]:
# set the path to the 2D segmentations ("mask") and the folder where the features should be saved
test_path = "./SHAPR/Blood_cell//mask/"
out_path =  "./Blood_cell/features/"

In [ ]:
# normalize and threshold the data using Otsu's method: 
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html
def norm_thres(data): 
    maxd = np.max(data)
    data = data / maxd
    data = np.nan_to_num(data)
    if np.min(data)  >0:
        thresh = threshold_otsu(data)
        binary = data > thresh
    else: 
        binary = data
    return binary*1.0

In [ ]:
files = os.listdir(test_path)
print("found", len(files), "files")
fdheader = ["fd"+str(i) for i in range(512)]

with open(out_path + 'mask_features.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    #create and combine the header
    header = ['filename_msk', "mean_msk", 'surface_msk', 'boundary_msk', 'roughtness_msk', 
              'convexity_msk', "moment_cr_msk", "moment_cc_msk", "gabor1_msk","gabor2_msk"]
    writer.writerow(header)
    
    for index, file in enumerate(files): 
        print(index, file)
        #read the data
        mask = np.nan_to_num(norm_thres(np.squeeze(imread(test_path + file))/255.))
        #calculate the boundary lenght
        msk = copy.deepcopy(mask)
        k = np.ones((3,3),dtype=int)
        msk = msk.astype(int)
        boundary = binary_dilation(msk==0, k) & msk
        boundary_len = np.sum(boundary)
        #calculate the area
        mask_vol = np.sum(msk)
        #calculate the roughness
        smoothed_mask = copy.deepcopy(gaussian_filter(mask,2))
        smoothed_mask = smoothed_mask > 0.5
        roughtness = np.sum(np.abs(smoothed_mask-mask))
        #calculate the convexity
        convexity = np.sum(convex_hull_image(mask)-mask)
        #calcualte the moments
        m_m = measure.moments(msk)
        cr_m = m_m[0, 1] / m_m[0, 0]
        cc_m = m_m[1, 0] / m_m[0, 0]
        
        #calcualte the gabor features
        gabor_msk1 = np.sum(gabor(msk,1)[0])
        gabor_msk2 = np.sum(gabor(msk,0.1)[0])
        #combine the features and write them to the .csv file
        feat = [file, np.mean(mask), mask_vol, boundary_len, roughtness,convexity, 
                cr_m, cc_m,  
                gabor_msk1, gabor_msk2]
    
        print("len features", len(feat), len(header))
        writer.writerow(feat)
